In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
response = requests.get('http://www.pinocchiospizza.net/menu.html')
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
def extract_toppings():
    table = []
    for row in soup.find("table", {"class": "toppingmenu"}).find_all('td'):
        table.append(row.text)
    return table

In [4]:
def extract_table(table_num):
    table = []
    # First found row is empty, so exclude it
    for row in soup.find_all("table", {"class": "foodmenu"})[table_num].find_all('tr')[1:]:
        table.append([element.text for element in row.find_all('td')])

    data = [row[1:] for row in table[1:]]  # Inner of `table` == [1:, 1:]
    columns = table[0][1:]  # First row of `table` == [0, 1:]
    index = [row[0] for row in table[1:]]  # First column of `table` == [1:, 0]
    
    return pd.DataFrame(data=data, columns=columns, index=index)

In [5]:
display(extract_toppings())

for i, name in zip(range(6), ['Regular', 'Sicilian', 'Subs', 'Pasta', 'Salads', 'Dinner Platters']):
    print("{}:".format(name))
    display(extract_table(i))

['Pepperoni',
 'Sausage',
 'Mushrooms',
 'Onions',
 'Ham',
 'Canadian Bacon',
 'Pineapple',
 'Eggplant',
 'Tomato & Basil',
 'Green Peppers',
 'Hamburger',
 'Spinach',
 'Artichoke',
 'Buffalo Chicken',
 'Barbecue Chicken',
 'Anchovies',
 'Black Olives',
 'Fresh Garlic',
 'Zucchini']

Regular:


,Small,Large
Cheese,12.70,17.95
1 topping,13.70,19.95
2 toppings,15.20,21.95
3 toppings,16.20,23.95
Special,17.75,25.95


Sicilian:


,Small,Large
Cheese,24.45,38.70
1 item,26.45,40.70
2 items,28.45,42.70
3 items,29.45,44.70
Special,30.45,45.70


...

,
Baked Ziti w/Mozzarella,6.50
Baked Ziti w/Meatballs,8.75
Baked Ziti w/Chicken,9.75


Salads:


,
Garden Salad,6.25
Greek Salad,8.25
Antipasto,8.25
Salad w/Tuna,8.25


Dinner Platters:


,
Garden Salad,Small $40.00 Large $65.00
Greek Salad,Small $50.00 Large $75.00
Antipasto,Small $50.00 Large $75.00
Baked Ziti,Small $40.00 Large $65.00
Meatball Parm,Small $50.00 Large $75.00
Chicken Parm,Small $55.00 Large $85.00


# DATABASE inserts

In [35]:
import sqlite3
conn = sqlite3.connect('db.sqlite3')
db = conn.cursor()

db.execute("""DELETE FROM orders_topping""")
db.execute("""DELETE FROM orders_menu""")

In [36]:
for topping in extract_toppings():
    sql = """
    insert into orders_topping (name)
    values (?)
    """
    db.execute(sql, (topping,))

In [37]:
def isFloat(str):
    try:
        float(str)
    except ValueError:
        return False
    return True

for i, kind in enumerate(['regular pizza', 'silician pizza', 'sub', 'pasta', 'salad']):
    df = extract_table(i)
    for vertical_field, row in df.iterrows():
        for horizontal_field, price in zip(row.index, row):
            sql = """
            insert into orders_menu (kind, name, size, price)
            values (?,?,?,?)
            """
            if isFloat(price):
                db.execute(sql, (kind, vertical_field, horizontal_field.strip(), price))
                # .strip() because for kind="pasta" "horizontal_field" becomes ' ' instead of ''

In [38]:
df = extract_table(5)
query_inputs = []
for vertical_field, row in df.iterrows():
    for horizontal_field, price in zip(row.index, row):
        # Remove blank elements after spliting single line
        batch = list(filter(lambda x: x!='', price.split(' ')))
        # Split line into 2 queries and ignore '$' sign
        query_inputs.append((vertical_field, batch[0], batch[1][1:]))
        query_inputs.append((vertical_field, batch[2], batch[3][1:]))
        
for name, size, price in query_inputs:
    sql = """
    insert into orders_menu (kind, name, size, price)
    values (?,?,?,?)
    """
    db.execute(sql, ('dinner platter', name, size, price))

In [39]:
conn.commit()
conn.close()